# Axial turbomachinery design
### turbomachinery: **axial compressor**
### target:
- $\beta = 1.45$
- max efficiency 
### constraints:
- $\dot{m} = 100 \frac{kg}{s}$
- $r_{tip}|_{max} = 0.9 m$
### inlet conditions:
- $P_{T0} = 1 bar$
- $T_{T0} = 300 K$

In [142]:
# importing libraries 
import numpy as np 
from thermoTransf.thermoProcess import *
from turboCoeff.similarity import *
import matplotlib.pyplot as plt 

# data
# constraints
mFlux = 100  # mass flux                [kg/s]
betaP = 1.45 # compression ratio        [--]
maxR  = 0.9  # maximum tip radius       [m]

# inlet values
Pt0 = 1e+5   # inlet total pressure     [Pa]
Tt0 = 300    # inlet total temperature  [K]

### Data and air properties
- air properties do not change in the stage
- $\beta$ allows using a single stage axial compressor 
    - it is worth noting we are close to the **efficiency limit** for an axial compressor stage
    - with respect to this $\eta \approx 0.9$
        - this can be seen as a good efficiency value
        - using a single stage compressor allows reducing **weight**
- $\eta \approx 0.9$ suggests:
    - $\chi \approx 0.55$ 
    - $\phi \approx 0.3$ 
- $V_{t0} \approx 0 \frac{m}{s}$, so mainly only axial flow speed $V_0$
- $r_{mean} \approx 0.75 m$
- it is assumed that the revolution velocity at the midspan of the blade is equal at the inlet and at the outlet: $U_0|_{midspan} = U_1|_{midspan} = U_{mean}$

In [143]:
# air properties allocation 
R     = 287.06                  # air gas constant Ru / Mm      [J/kg K]
gamma = 1.4                     # specific heat ratio           [--]
cP    = gamma / (gamma - 1) * R # specific heat ratio @ P cost  [J/kg K]
cV    = cP - R                  # specific heat ratio @ V cost  [J/kg K]

# stage hypothesis
eta = 0.9    # stage efficiency 
rD  = 0.55   # reaction degree 
psi = 0.3    # 
phi = 0.6    #
rMean = 0.2 # stage mean radius -> radius @ inlet blade midspan  
Vt0 = 0      # inlet aboslute tangential speed

work computation

In [144]:
# ideal compression work 
Lis = cP * Tt0 * (betaP**((gamma-1)/gamma) - 1)
# real compression work given eta 
L = Lis / eta
# total temperature computation
Tt1is = Lis / cP + Tt0
Tt1r = L / cP + Tt0 
# print data 
print('Lis   = {0:>10.2f} J/kg'.format(Lis))
print('Tt1is = {0:>10.2f} K'.format(Tt1is))
print('L     = {0:>10.2f} J/kg'.format(L))
print('Tt1r  = {0:>10.2f} K'.format(Tt1r))

Lis   =   33758.53 J/kg
Tt1is =     333.60 K
L     =   37509.48 J/kg
Tt1r  =     337.33 K


revolution speed and angular speed computation

In [145]:
# mean section revolution speed  
Umean = np.sqrt(Lis / psi) 
# angular speed 
omega = Umean / rMean
# rpm 
n = omega * 60 / (2 * np.pi)
# print data
print('Umean = {0:>10.2f} m/s \nomega = {1:>10.2f} rad/s\nn     = {2:>7.0f}    rpm'.format(Umean, omega, n))  

Umean =     335.45 m/s 
omega =    1677.26 rad/s
n     =   16017    rpm


axial inlet velocity computation

In [146]:
# axial inlet velocity 
Va0 = phi * Umean
print('Va0 = {0:>10.2f} m/s'.format(Va0))

Va0 =     201.27 m/s


tangential velocity computation

In [147]:
# under Umean assumption & Vt0 assumption 
lam = 2 * Lis / Umean**2
Vinft = Umean * lam / 4 + Vt0
Vt1 = 2 * Vinft - Vt0   
print('Vinft = {0:>10.2f} m/s'.format(Vinft))
print('Vt1   = {0:>10.2f} m/s'.format(Vt1))

Vinft =      50.32 m/s
Vt1   =     100.64 m/s


inlet quantities

In [148]:
# main inlet quantities computation
Wa0 = Va0
Wt0 = Vt0 - Umean
W0 = np.sqrt(Wa0**2 + Wt0**2)
V0 = np.sqrt(Va0**2 + Vt0**2)
# thermodynamics
T0 = Tt0 - V0**2 / (2*cP)
a0 = np.sqrt(gamma * R * T0)
M0 = V0 / a0
P0 = Pt0 / (1 + (gamma - 1)/2 * M0**2)**(gamma/(gamma-1))
rho0 = P0 / (R * T0)
# print data 
print('Wa0  = {0:>10.2f} m/s'.format(Wa0))
print('Wt0  = {0:>10.2f} m/s'.format(Wt0))
print('W0   = {0:>10.2f} m/s'.format(W0))
print('V0   = {0:>10.2f} m/s'.format(V0))
print('T0   = {0:>10.2f} K'.format(T0))
print('a0   = {0:>10.2f} m/s'.format(a0))
print('M0   = {0:>10.2f} '.format(M0))
print('P0   = {0:>10.2f} bar'.format(P0/1e+5))
print('rho0 = {0:>10.2f} kg/m3'.format(rho0))

Wa0  =     201.27 m/s
Wt0  =    -335.45 m/s
W0   =     391.20 m/s
V0   =     201.27 m/s
T0   =     279.84 K
a0   =     335.36 m/s
M0   =       0.60 
P0   =       0.78 bar
rho0 =       0.98 kg/m3


inlet blade height

In [149]:
# inlet blade height 
b0 = mFlux / (rho0 * 2 * np.pi * rMean * Va0)
print('b0 = {0:>10.2f} m'.format(b0))

b0 =       0.41 m


outlet quantities

In [150]:
# main outlet quantities 
Va1 = Va0 
Wa1 = Va1 
Wt1 = Vt1 - Umean
V1 = np.sqrt(Va1**2 + Vt1**2)
W1 = np.sqrt(Wa1**2 + Wt1**2)
# thermodynamics
T1 = Tt1is - V1**2 / (2*cP)
a1 = np.sqrt(gamma * R * T1)
M1 = V1 / a1
P1 = P0 * (T1/T0)**(gamma/(gamma-1))
Pt1is = P1 * (Tt1is/T1)**(gamma/(gamma-1))
rho1 = P1 / (R * T1) 
# print data 
print('Wa1   = {0:>10.2f} m/s'.format(Wa1))
print('Wt1   = {0:>10.2f} m/s'.format(Wt1))
print('W1    = {0:>10.2f} m/s'.format(W1))
print('V1    = {0:>10.2f} m/s'.format(V1))
print('T1    = {0:>10.2f} K'.format(T1))
print('a1    = {0:>10.2f} m/s'.format(a1))
print('M1    = {0:>10.2f} '.format(M1))
print('P1    = {0:>10.2f} bar'.format(P1/1e+5))
print('rho1  = {0:>10.2f} kg/m3'.format(rho1))
print('Pt1is = {0:>10.2f} bar'.format(Pt1is/1e+5))

Wa1   =     201.27 m/s
Wt1   =    -234.82 m/s
W1    =     309.27 m/s
V1    =     225.03 m/s
T1    =     308.40 K
a1    =     352.05 m/s
M1    =       0.64 
P1    =       1.10 bar
rho1  =       1.24 kg/m3
Pt1is =       1.45 bar


outlet blade height computation

In [151]:
# outlet blade height
b1 = mFlux / (rho1 * 2 * np.pi * rMean * Va1)
print('b1 = {0:>10.2f} m'.format(b1))

b1 =       0.32 m


### Blade design 
#### model: **free vortex** $\frac{\partial r V_t}{\partial r} = 0$
### hypothesis:
- $\frac{\partial r h_1}{\partial r} = 0$

- $\frac{\partial r V_{a1}}{\partial r} = 0$

- $s$ evolution along the blade span